In [10]:
import pandas as pd
import numpy as np
import ensemble_boxes 
from pycocotools.coco import COCO
import itertools
import os
from tqdm import tqdm

In [45]:
submission_df

,PredictionString,image_id
0,7 0.8575605 602.07666 518.8392 957.3976 1022.0...,test/0000.jpg
1,5 0.71651125 130.04492 0.008392334 492.8197 26...,test/0001.jpg
2,1 0.81122565 75.83356 271.3814 427.62433 601.1...,test/0002.jpg
3,9 0.7855117 143.49927 262.19116 913.0812 824.0...,test/0003.jpg
4,1 0.8159588 196.33911 249.19266 875.6224 779.1...,test/0004.jpg
...,...,...
4866,0 0.7796096 292.21106 346.94254 670.067 687.37...,test/4866.jpg
4867,3 0.7924054 415.67303 326.54025 782.65845 702....,test/4867.jpg
4868,7 0.86894023 112.02197 448.36627 758.4912 836....,test/4868.jpg
4869,0 0.763658 2.0211182 239.45267 1025.8834 938.3...,test/4869.jpg


'deta-swin-large_0_img_size_720'

In [54]:
mode = 'test'
transforms = ['identity', 'hflip']
prefix = '1022'

submission_fold_path = f'result/TTA/{mode}'
submission_file_format = 'deta-swin-large_0_img_size_720_{transform}.csv'
output_name_format = prefix + '_' + submission_file_format[0:30] + '_{algorithm}_thres_{iou_threshold:.1f}.csv'
output_fold_path = f'result/ensemble-1fold/{mode}'
algorithms = ['nms', 'weighted_boxes_fusion']
iou_thresholds = np.arange(0.7, 0.91, 0.1).tolist()

output_path_format = output_fold_path + '/' + output_name_format
annotation_path = '/home/jin/project/Object detection/data/dataset/test.json'

submissions_df = []
for transform in transforms:
    submission_file_path = os.path.join(submission_fold_path, 
                                        submission_file_format.format(mode=mode, transform=transform))
    submission_df = pd.read_csv(submission_file_path)
    submissions_df.append(submission_df)

image_ids = submissions_df[0]['image_id'].tolist()
coco = COCO(annotation_path)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [55]:
def get_box_score_label(submission_df, image_id, image_info):
    boxes_list = []
    scores_list = []
    labels_list = []

    # 각 submission file 별로 prediction box좌표 불러오기
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()

        if len(predict_list)==0 or len(predict_list)==1:
            continue

        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []

        for box in predict_list[:, 2:6].tolist():
            # box의 각 좌표를 float형으로 변환한 후 image의 넓이와 높이로 각각 정규화
            image_width = image_info['width']
            image_height = image_info['height']

            box[0] = float(box[0]) / image_width
            box[1] = float(box[1]) / image_height
            box[2] = float(box[2]) / image_width
            box[3] = float(box[3]) / image_height

            box = np.clip(box, 0, 1).tolist()
            box_list.append(box)

        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))

    return boxes_list, scores_list, labels_list


def get_prediction(boxes, scores, labels, image_width, image_height):
    prediction_string = ""
    for box, score, label in zip(boxes, scores, labels):
        xmin, ymin = box[0] * image_width, box[1] * image_height
        xmax, ymax = box[2] * image_width, box[3] * image_height
        prediction_string += f'{label:.0f} {score:.5f} {xmin:.2f} {ymin:.2f} {xmax:.2f} {ymax:.2f} '
    return prediction_string

In [57]:
def ensemble(algorithms, iou_thresholds, image_ids, submissions_df, output_path_format):
    conf = list(enumerate(list(itertools.product(algorithms, iou_thresholds)))) # ensemble할 조건 리스트
    results = [{'prediction_strings': [], 'file_names': []} for i in range(len(conf))] # 결과 저장 리스트 선언

    # 각 image id 별로 submission file에서 box좌표 추출
    for i, image_id in enumerate(tqdm(image_ids)):    
        prediction_string = ''
        boxes_list = []
        scores_list = []
        labels_list = []

        image_info = coco.loadImgs(int(image_id[6:-4]))[0]
        image_width = image_info['width']
        image_height = image_info['height']

        boxes_list, scores_list, labels_list = get_box_score_label(submissions_df, image_id, image_info)
        
        # 예측 box가 있다면 ensemble 수행
        if len(boxes_list):
            # ensemble에 필요한 인자: [box의 lists, confidence score의 lists, label의 list, iou에 사용할 threshold]
            for j, (algorithm, iou_threshold) in conf:
                boxes, scores, labels = getattr(ensemble_boxes, algorithm)(boxes_list, scores_list, labels_list, iou_thr=iou_threshold)
                prediction_string = get_prediction(boxes, scores, labels, image_width, image_height)

                results[j]['prediction_strings'].append(prediction_string)
                results[j]['file_names'].append(image_id)    

    # 결과 저장하기
    num_file = len(submissions_df)
    os.makedirs(output_fold_path, exist_ok=True)

    for i, (algorithm, iou_threshold) in conf:
        output_path = output_path_format.format(algorithm=algorithm, iou_threshold=iou_threshold)

        submission = pd.DataFrame()
        submission['PredictionString'] = results[i]['prediction_strings']
        submission['image_id'] = results[i]['file_names']

        submission.to_csv(output_path, index=False)
        submission.head()

100%|██████████| 4871/4871 [00:51<00:00, 95.50it/s] 
